# Intelligent Systems 2023: 3rd practical assignment
## Informed Search

Your name: Amund Strøm

Your VUNetID: ast101

If you do not provide your name and VUNetID we will not accept your submission.

### Learning objectives
At the end of this exercise you should be able to understand the implementations of the basic algorithms for informed search. You should be able to:

1. Understand the algorithms (be able to explain in your own words)
2. Follow the individual steps of the algorithms
3. Make small modifications of the code to see the effect on the search algorithms
4. Make small adaptations to the algorithm to study the computational properties

### Practicalities

Follow this Notebook step-by-step.

Of course, you can do the exercises in any Programming Editor of your liking.
But you do not have to. Feel free to simply write code in the Notebook. Please use your studentID+Assignment2.ipynb as the name of the
Notebook.  

Note: unlike the courses dedicated to programming we will not evaluate the style of the programs. But we will, however, test your programs on other data that we provide, and your program should give the correct output to the test-data as well.

As was mentioned, the assignment is graded as pass/fail. To pass you need to have either a full working code or an explanation of what you tried and what didn't work for the tasks that you were unable to complete (you can use multi-line comments or a text cell).

## Initialising

First, let us make sure the necessary packages are installed, and imported. Run the following code:

In [46]:
import sys
#!{sys.executable} -m pip install numpy
import datetime
import functools
import heapq
import numpy as np
from numpy import random
from collections import deque

from utils import *

# This might produce a warning that numpy is already installed.

## Starting your first game

In [47]:
HAND_SIZE = 10      #TODO: replace with your desired hand size (3, 5 or 10)
STUDENT_NUMBER = 2822237 #TODO: replace with your own student number

With the constant HAND_SIZE we decide how many cards we want in our hand.
By default it is set to 5, you can change it to any hand size,
do keep in mind that the higher the number of cards in your hand,
the more branches in the search tree there are, and the longer it will take to run.

Your student number is used to set a random seed.
There are situations imaginable where you want a pseudo random selection (for example when debugging it's nice to always work with the same values)
In short, the seed ensures that you get a pseudo random distribution that will always be the same when you re-run the code.
It is a random distribution because you don't have to hard code them in yourself,
but it is not random in the sense that the next time you run it you get different cards!
For more information on pseudo random number generators,
check out https://www.geeksforgeeks.org/pseudo-random-number-generator-prng/.

### Updated representation

You may wonder how the cards for this game are represented.
Go to utils and find out in which variable this information is found, print this variable below:

In [48]:
print(representation)


        h  d  s  c
ace  [[ 0  1  2  3]  = 11pts     h = hearts
ten   [ 4  5  6  7]  = 10pts     d = diamonds
king  [ 8  9 10 11]  = 4pts      s = spades
queen [12 13 14 15]  = 3pts      c = clubs
jack  [16 17 18 19]] = 2pts
For example: '10' is 'king of spades'



Note that cards have values now w.r.t. the game from Assignment 2

### Some support functions

Next, there are some functions we need for the implementation. Try to get the gist of what they do, but if you do not understand fully, don't worry. The first one returns the points value of a card, the second tests whether a move is valid (so whether a card follows suit, or has the same value). The third is a helper function that checks whether two cards have the same suit, and the fourth function checks whether two cards have the same value. The last one makes a random choice of cards in the hands.

We don't expect you to fully understand the coding behind these functions, however, if you are interested you might find the following link useful: https://www.programiz.com/python-programming/matrix. Take a look at how Python Matrices are created and how to access rows and columns.

In [49]:
def get_points(cardA):
    g = np.arange(20).reshape(5, 4) #this produces the same grid as the representation
    if (np.any(g[0] == cardA)): #Ace (11)
        return 11
    elif (np.any(g[1] == cardA)): #Ten (10)
        return 10
    elif (np.any(g[2] == cardA)): #King (4)
        return 4
    elif (np.any(g[3] == cardA)): #Queen (3)
        return 3
    elif (np.any(g[4] == cardA)): #Jack (2)
        return 2

def valid_move(cardA, cardB):
    #print("validMove: comparing " + str(cardA) + " to " + str(cardB)) UNCOMMENT THIS TO SEE WHICH CARDS ARE BEING COMPARED
    g = np.arange(20).reshape(5, 4) #this produces the same grid as the representation, for the purpose of checking moves
    if check_value(cardA, cardB, g):
        return True
    elif check_suit(cardA, cardB, g):
        return True
    else:
        #print("validMove: No move found")
        return False

def check_suit(cardA, cardB, grid):
    r, c = grid.shape
    for i in range(c):
        if np.any(grid[:, i] == cardA) and np.any(grid[:, i] == cardB):
            return True

def check_value(cardA, cardB, grid):
    r, c = grid.shape
    for i in range(r):
        if np.any(grid[i] == cardA) and np.any(grid[i] == cardB):
            return True

def pick_cards(seed, size):
    random.seed(seed)
    cards = np.random.choice(20, (size*2), replace = False)
    leftHand = cards[:size]
    rightHand = cards[size:]
    return (leftHand, rightHand)

### Breadth, and Depth First Search

From assignment 1 we are already familiar with the Breadth First and Depth First search algorithms. See the code below:

In [50]:
def breadth_first_tree_search(problem):
    """
    Search the shallowest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument fringe should be an empty queue.
    Repeats infinitely in case of loops.
    """
    counter = 0
    fringe = deque([Node(problem.initial)])  # FIFO queue
    while fringe:
        node = fringe.popleft()
        counter += 1
        if problem.goal_test(node.state):
            print("###########")
            print("success!")
            print("solution: {}".format(node.solution()))
            print("{} Nodes generated".format(counter))
            return node
        fringe.extend(node.expand(problem))
    print("###########")
    print("unfortunately no solution has been found!")
    print("{} Nodes generated".format(counter))
    return None

In [51]:
def depth_first_tree_search(problem):
    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument fringe should be an empty queue.
    Repeats infinitely in case of loops.
    """
    counter = 0
    fringe = [Node(problem.initial)]  # Stack
    while fringe:
        node = fringe.pop()
        counter += 1
        if problem.goal_test(node.state):
            print("###########")
            print("succes!")
            print("solution: {}".format(node.solution()))
            print("{} Nodes generated".format(counter))
            return node
        fringe.extend(node.expand(problem))
    print("###########")
    print("unfortunately no solution has been found!")
    print("{} Nodes generated".format(counter))
    return None

### Preparing the game
To run a search, we need to define an initial state. Run the cells below to generate hands automatically, or define a custom set of hands. Please note that if you use a custom set of hands, you need to replace 'leftHand' and 'rightHand' with 'customLeftHand' and 'customRightHand' at gamestate initialisation.

The cell below generates the cards for your left and right hand.

In [52]:
leftHand, rightHand = pick_cards(STUDENT_NUMBER, HAND_SIZE)
#customLeftHand = []
#customRightHand = []

print("Left hand: {}".format(leftHand)) #or customLeftHand
print("Right hand: {}".format(rightHand)) #or customRightHand

Left hand: [ 7 12  4  2  5 10  1 15 16 14]
Right hand: [ 8  3  6 19 18 17  0  9 13 11]


Now we define an initial state, and the problem.

In [53]:
initialState = GameState(leftHand, rightHand, True, True)
initialState.printState()

p = Problem(initialState)
print("There are {} points needed to win(reach goal state)".format(p.winPoints))
print("There are {} points in the game".format(sum(p.initial.allCardPoints)))


------
GameState: Printing state: 
Left hand: [ 7 12  4  2  5 10  1 15 16 14]
Right hand: [ 8  3  6 19 18 17  0  9 13 11]
Do we play from left hand to get to next state? True
Points scored: 0
All cards: [7, 12, 4, 2, 5, 10, 1, 15, 16, 14, 8, 3, 6, 19, 18, 17, 0, 9, 13, 11]
All card points: [10, 3, 10, 11, 10, 4, 11, 3, 2, 3, 4, 11, 10, 2, 2, 2, 11, 4, 3, 4]
------
There are 66 points needed to win(reach goal state)
There are 120 points in the game


## Task 1:

Run Breadth and Depth first search and implement a counter in the algorithm above that prints the number of nodes generated (as you did in the previous assignment, although now to print you must uncomment the print statements instead of formulating your own).

In [54]:
startTime = datetime.datetime.now()
s=breadth_first_tree_search(p)
endTime = datetime.datetime.now()
duration = endTime - startTime
print("The Breadth First Search Took {} Microseconds".format(duration.microseconds))
print("BFS PATH_COST: {}".format(s.path_cost))

startTime = datetime.datetime.now()
s=depth_first_tree_search(p)
endTime = datetime.datetime.now()
duration = endTime - startTime
print("The Depth First Search Took {} Microseconds".format(duration.microseconds))
print("DFS PATH_COST: {}".format(s.path_cost))


###########
success!
solution: [7, 3, 2, 6, 4, 0, 12]
5041 Nodes generated
The Breadth First Search Took 101492 Microseconds
BFS PATH_COST: 220
###########
succes!
solution: [14, 13, 15, 11, 10, 9, 1, 0, 16, 17, 5, 6]
13 Nodes generated
The Depth First Search Took 41468 Microseconds
DFS PATH_COST: -42


### Greedy Best First Search

See the code below for the Greedy Best First Search Algorithm

In [55]:
def greedy_best_first_search(problem, f, display=True):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; """
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    counter = 0
    while frontier:
        node = frontier.pop()
        counter += 1
        if problem.goal_test(node.state):
            if display:
                print("Search succesful!")
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
                print("solution: {}".format(node.solution()))
                print("{} Nodes generated".format(counter))
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    print("Search failed")
    print("{} Nodes generated".format(counter))
    return None

## Task 2

Run A Greedy Best First Search and implement a counter in the algorithm above that prints the number of nodes generated (as you did in the previous assignment, although now to print you must uncomment the print statements instead of formulating your own).

In [56]:
startTime = datetime.datetime.now()
greedy_best_first_search(p, p.h, True)
endTime = datetime.datetime.now()
duration = endTime - startTime
print("The Greedy Best First Search Took {} Microseconds".format(duration.microseconds))

Search succesful!
7 paths have been expanded and 23 paths remain in the frontier
solution: [2, 3, 1, 0, 4, 6, 7]
8 Nodes generated
The Greedy Best First Search Took 47700 Microseconds


## Task 3

In what way do Depth First Search, Breadth First Search and Greedy Best First Search (using the distance to the goal as heuristic) differ?

Return a (brief) report to the multi line string below, take into account the number of nodes counted from the counter you implemented:


In [57]:
myReport1 = """
The different search algorithms differ in their strategies for exploring and selecting nodes in the tree.

Breadth First Search will explore all the neighbor nodes at the present depth prior to moving on to nodes at the next depth level. This strategy makes BFS complete and 
ensures that it will find the shortest path to the goal state, at the cost of exploring a lot of nodes.

Depth First Search explores as deep as possible along each branch before backtracking to the latest node with a different path. DFS is neither complete or optimal, meaning
it is not guaranteed that it will find a goal state if there exists one and it is not guaranteed that it will find the shortest path. But in the case of Schnapsen it will
always be faster than BFS since the goal state exists deeper into the tree.

Gready Best First Search uses a heuristic function to estimate the cost to reach the goal from a given node. It always explores the node that appears to be the most promising
based on the heuristic. In our case the heuristic is the distance to the goal, based on points. GBFS is also neither complete or optimal, it may repeat nodes that aleardy have
been explored and will end up in a cycle. It may also lead to a false start, choosing the wrong branch at the very start.

In the case of the current game state, we get these results:
    - BFS generated 5041 nodes, solution [7, 3, 2, 6, 4, 0, 12]
    - DFS generated 13 nodes, solution [14, 13, 15, 11, 10, 9, 1, 0, 16, 17, 5, 6]
    - GBFS generated 8 nodes, solution [2, 3, 1, 0, 4, 6, 7]

We see that BFS is by far the slowset but finds an optimal solution. DFS is much faster but does not find an optimal soultion. GBFS is the fastest and in this case finds an
optimal soultion. We see that the soultion consists of the heighest valued cards, which is choosen by the heuristic.
"""

## Task 4

Run a Greedy Best First Search (again) but this time print the path cost (uncomment the print statement and fill in the right variable.)

(Hint: Go into utils.py, in class Node you can see a variable that should help you. Remember that a -successful- search returns a Node which is saved as 'search' here)

In [58]:
search = greedy_best_first_search(p, p.h)

print("Path cost: {}".format(search.path_cost))

Search succesful!
7 paths have been expanded and 23 paths remain in the frontier
solution: [2, 3, 1, 0, 4, 6, 7]
8 Nodes generated
Path cost: 228


### A* Search algorithm

See the code for the A star search below:

In [59]:
def astar_search(problem, h=None, display=True):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    return greedy_best_first_search(problem, lambda n: n.path_cost + h(n), display)

## Task 5

Consider the the solution of a greedy best first search and an a star search:

In [60]:
print("Greedy Best First Search: ")
gbf=greedy_best_first_search(p, p.h, True)
print("Path cost: {}".format(gbf.path_cost))

print("A Star Search: ")
ast=astar_search(p)
print("Path cost: {}".format(ast.path_cost))


Greedy Best First Search: 
Search succesful!
7 paths have been expanded and 23 paths remain in the frontier
solution: [2, 3, 1, 0, 4, 6, 7]
8 Nodes generated
Path cost: 228
A Star Search: 


TypeError: 'NoneType' object is not callable

Explain in your own words what the difference between the two algorithms is, and why the solutions differ. Return your explanation to the multi line string below:

In [ ]:
myReport2 = """
Both Gready Best First Search and A* Algorithm uses heuristics to choose which nodes to explore, but they differ in how they use the evaluation function.

Gready Best First Search only consieders the value from the heuristic to choose the next node. The evaluation function is used to calculate the node with the most promising
value, and looks like f(n) = h(n), where h(n) is the heuristic value estimating the cost from a node n to the goal state.

A* Algorithm considers both the value from the heuristic plus the value it cost to get to that node. It selects the node that minimizes the evaluation function f(n) = g(n) + h(n),
where g(n) is the cost it takes to reach a specific node n from the start state. h(n) is the same as before, the heuristic value estimating the cost from a node n to the goal state.

I had some errors with the A* function, and could therefore compare the different solutions.
"""

## Final Task: Collect all the results

Uncomment and run this cell (and all the cells above) to generate the text file that you have to hand in together with the notebook on canvas!

In [ ]:
exportToText("assignment3.txt",STUDENT_NUMBER, leftHand, rightHand, myReport1, myReport2 )